In [1]:
import os
from glob import glob
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [5]:
parent_folder = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\data\SkeletonData\SkeletonData\Simplified"  # Path to folder containing the data
num_timesteps = 100  # number of timesteps to set in training data
paths = glob(parent_folder + "/*.txt")

In [6]:
from sklearn.preprocessing import StandardScaler
def flip_coords(positions_df, body_parts):  # flip x-coordinates in sample around central x-coord as well as change right identities to left
    # Flip x-coords around central coordinate
    positions_array = positions_df.to_numpy()
    x_coords = positions_df.iloc[:, ::3].to_numpy()
    x_coords_flattened = np.reshape(x_coords.flatten(), (x_coords.size, 1))
    sc = StandardScaler()
    x_coords_flipped = sc.fit_transform(x_coords_flattened)
    x_coords_flipped = -x_coords_flipped
    x_coords_flipped = sc.inverse_transform(x_coords_flipped)
    x_coords_flipped = np.reshape(x_coords_flipped, x_coords.shape)
    positions_array[:, ::3] = x_coords_flipped
    new_positions_df = pd.DataFrame(positions_array)


    # Change right identities to left
    col_list = list(new_positions_df)
    col_list_copy = col_list.copy()
    for b, this_part in enumerate(body_parts):
        if this_part.endswith('Right'):
            right_index = b * 3  # index of x position for right body part
            left_index = body_parts.index(this_part.split("_")[0] + "_Left") * 3
            col_list[right_index:right_index + 3], col_list[left_index:left_index + 3] = col_list[
                                                                                         left_index:left_index + 3], col_list[
                                                                                                                     right_index:right_index + 3]
    new_positions_df = new_positions_df.iloc[:, col_list]
    new_positions_df.columns = col_list_copy
    return new_positions_df


In [8]:
# Go through all files and compile into one dataframe, combining left and right variants into the same category
train_df = pd.DataFrame()  # holds sequences of num_timesteps timepoints
count = 0
body_parts = ["Spine_Base", "Spine_Mid", "Neck", "Head", "Shoulder_Left", "Elbow_Left", "Wrist_Left", "Hand_Left", "Shoulder_Right", "Elbow_Right", "Wrist_Right", "Hand_Right", "Hip_Left", "Knee_Left", "Ankle_Left", "Foot_Left", "Hip_Right", "Knee_Right", "Ankle_Right", "Foot_Right", "Spine_Shoulder", "Tip_Left", "Thumb_Left", "Tip_Right", "Thumb_Right"
]
gestures = ["EFL", "EFR", "SFL", "SFR", "SAL", "SAR", "SFE", "STL", "STR"]
gestures_combined = ["EF", "SF", "SA", "SFE", "ST"]

patient_ids = []
for t, this_path in tqdm(enumerate(paths)):
    fname, ext = os.path.splitext(os.path.split(this_path)[1])
    if ext != ".txt":
        continue

    patient_id, date_id, og_gesture_label, repetition_id, correct_label, position = fname.split("_")
    if int(correct_label) == 3:  # Poorly performed gestures
        continue
    if patient_id not in patient_ids:
        patient_ids.append(patient_id)

    joint_positions = pd.read_csv(this_path, header=None)

    # Flip right-side variants to left-side variants
    gesture_name = gestures[int(og_gesture_label)]
    if gesture_name.endswith("R"):
        joint_positions = flip_coords(joint_positions, body_parts)
    # Change gesture labels
    if gesture_name.endswith("R") or gesture_name.endswith("L"):
        gesture_name = gesture_name[:-1]
    gesture_label = str(gestures_combined.index(gesture_name))

    n_rows = len(joint_positions)
    joint_positions['patient_id'] = [int(patient_id) for _ in range(n_rows)]
    joint_positions['gesture_label'] = [int(gesture_label) for _ in range(n_rows)]
    joint_positions['repetition_id'] = [int(repetition_id) for _ in range(n_rows)]
    joint_positions['correct_label'] = [int(correct_label) - 1 for _ in range(n_rows)]
    joint_positions['position'] = [position for _ in range(n_rows)]
    joint_positions['frame'] = [_ for _ in range(n_rows)]
    joint_positions['number'] = [count for _ in range(n_rows)]
    joint_positions['og_gesture_label'] = [og_gesture_label for _ in range(n_rows)]

    n_frames = len(joint_positions)
    if n_frames < num_timesteps:  # play sequence reversed until n_frames = 100
        joint_positions_reversed = joint_positions[::-1]
        for n_recurr in range(num_timesteps // n_frames):
            joint_positions = pd.concat([joint_positions, joint_positions_reversed], ignore_index=True)
            joint_positions_reversed = joint_positions_reversed[::-1]
        joint_positions = joint_positions[:num_timesteps]
        joint_positions['frame'][:] = range(num_timesteps)
        train_df = pd.concat([train_df, joint_positions], ignore_index=True)
        count += 1
    else:
        n_clips = n_frames // num_timesteps
        n_leftover_frames = n_frames % num_timesteps
        separation = n_leftover_frames // n_clips
        for this_clip in range(n_clips):
            start_index = this_clip*num_timesteps + separation*this_clip
            this_joint_positions = joint_positions.loc[start_index:start_index+num_timesteps - 1]  # pd dataframes include both start and end points
            this_joint_positions['frame'][:] = range(len(this_joint_positions))
            this_joint_positions['number'][:] = [count for _ in range(len(this_joint_positions))]
            train_df = pd.concat([train_df, this_joint_positions], ignore_index=True)
            count += 1


13it [00:00, 120.32it/s]C:\Users\rfgla\AppData\Local\Temp\ipykernel_16412\2898842251.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_joint_positions['frame'][:] = range(len(this_joint_positions))
C:\Users\rfgla\AppData\Local\Temp\ipykernel_16412\2898842251.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_joint_positions['number'][:] = [count for _ in range(len(this_joint_positions))]
26it [00:00, 99.61it/s] C:\Users\rfgla\AppData\Local\Temp\ipykernel_16412\2898842251.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation

In [11]:
# Organize into folders based on gesture class, with y = correct label
save_folder = r"C:\Users\rfgla\Documents\Ray\telerehab_exercise_feedback\data\skeleton_data_gestures_combined_correct_sorted_LOSO"
if not os.path.exists(save_folder):
    os.mkdir(save_folder)
gestures = ["EF", "SF", "SA", "SFE", "ST"]
for this_patient in patient_ids:
    this_patient = int(this_patient)
    patient_test_df = train_df[train_df['patient_id'] == this_patient]
    patient_train_df = train_df[train_df['patient_id'] != this_patient]
    patient_folder = os.path.join(save_folder, f"patient_{this_patient}")
    if not os.path.exists(patient_folder):
        os.mkdir(patient_folder)
    for this_gesture in range(len(np.unique(train_df['gesture_label']))):
        gesture_folder = os.path.join(patient_folder, gestures[this_gesture])
        if not os.path.exists(gesture_folder):
            os.mkdir(gesture_folder)

        this_test_df = patient_test_df[patient_test_df['gesture_label'] == this_gesture]
        this_train_df = patient_train_df[patient_train_df['gesture_label'] == this_gesture]

        test_y_df = this_test_df[this_test_df['frame'] == 0]
        test_y_df = test_y_df['correct_label']
        train_y_df = this_train_df[this_train_df['frame'] == 0]
        train_y_df = train_y_df['correct_label']

        this_train_df.to_csv(os.path.join(gesture_folder, "train_x.csv"), header=None, index=False)
        train_y_df.to_csv(os.path.join(gesture_folder, "train_y.csv"), header=None, index=False)
        test_y_df.to_csv(os.path.join(gesture_folder, "test_y.csv"), header=None, index=False)
        this_test_df.to_csv(os.path.join(gesture_folder, "test_x.csv"), header=None, index=False)